# scEU-seq organoid

This tutorial uses the intestine organoid data from Battich, et al (2020). This tutorial is the second one of the two tutorials for demonstrating how dynamo can use used to analyze the scEU-seq data. Please refer the [cell cycle](https://dynamo-release.readthedocs.io/en/latest/scEU_seq_rpe1_analysis_kinetic.html) tutorial for details on how to analyze the cell cycle dataset.


In [1]:
import warnings
warnings.filterwarnings('ignore')

import dynamo as dyn
import anndata
import pandas as pd
import numpy as np
import scipy.sparse

from anndata import AnnData
from scipy.sparse import csr_matrix

dyn.get_all_dependencies_version()

|-----> setting visualization default mode in dynamo. Your customized matplotlib settings might be overritten.


package,umap-learn,pynndescent,python-igraph,numdifftools,seaborn,statsmodels,numba,scikit-learn,dynamo-release,cvxopt,pandas,scipy,numpy,colorcet,networkx,pre-commit,loompy,openpyxl,matplotlib,get-version,tqdm,setuptools
version,0.5.3,0.5.7,0.9.11,0.9.40,0.11.2,0.13.2,0.55.1,1.0.2,1.1.0,1.3.0,1.4.2,1.7.3,1.21.5,2.0.6,2.7.1,2.19.0,3.0.7,3.0.9,3.5.1,3.5.4,4.64.0,61.2.0


## Load data

In [9]:
organoid = dyn.sample_data.scEU_seq_organoid()
# from anndata import read_h5ad
# organoid = read_h5ad(filename = "data/rpe1.h5ad")
# organoid.var_names_make_unique()

In [10]:
organoid

AnnData object with n_obs × n_vars = 5422 × 11848
    obs: 'Plate_Id', 'Condition_Id', 'Well_Id', 'RFP_log10_corrected', 'GFP_log10_corrected', 'Cell_cycle_possition', 'Cell_cycle_relativePos', 'exp_type', 'time'
    var: 'Gene_Id'
    layers: 'sl', 'su', 'ul', 'uu'

In [11]:
# mapping:
cell_mapper = {
    '1': 'Enterocytes',
    '2': 'Enterocytes',
    '3': 'Enteroendocrine',
    '4': 'Enteroendocrine progenitor',
    '5': 'Tuft cells',
    '6': 'TA cells',
    '7': 'TA cells',
    '8': 'Stem cells',
    '9': 'Paneth cells',
    '10': 'Goblet cells',
    '11': 'Stem cells',
 }

organoid.obs['cell_type'] = organoid.obs.som_cluster_id.map(cell_mapper).astype('str')


AttributeError: 'DataFrame' object has no attribute 'som_cluster_id'

## typical dynamo analysis workflow


In [ ]:
dyn.pl.basic_stats(organoid)

In [ ]:
organoid

In [ ]:
organoid.obs

In [ ]:
organoid.obs.groupby(['exp_type', 'time']).agg('count')

In [ ]:
adata = organoid.copy()
adata.obs.time = adata.obs.time.astype('str')
adata.obs.loc[adata.obs['time'] == 'dmso', 'time'] = -1
adata.obs['time'] = adata.obs['time'].astype(float)
adata = adata[adata.obs.time != -1, :]
adata = adata[adata.obs.exp_type == 'Pulse', :]
adata.layers['new'], adata.layers['total'] = adata.layers['ul'] + adata.layers['sl'], adata.layers['su'] + adata.layers['sl'] + adata.layers['uu'] + adata.layers['ul']
del adata.layers['uu'], adata.layers['ul'], adata.layers['su'], adata.layers['sl']
adata = dyn.pp.recipe_monocle(adata, n_top_genes=1000, total_layers=False)
dyn.pl.basic_stats(adata)
dyn.pl.show_fraction(organoid)

In [ ]:
adata.obs.time = adata.obs.time/60

In [ ]:
adata.obs.time  = adata.obs.time.astype('float')
dyn.tl.dynamics(adata, model='deterministic', tkey='time', assumption_mRNA='ss')

dyn.tl.reduceDimension(adata)

In [ ]:
dyn.tl.cell_velocities(adata, ekey='M_t', vkey='velocity_T', enforce=True)


In [ ]:
adata.obsm['X_umap_ori'] = adata.obs.loc[:, ['rotated_umap1', 'rotated_umap2']].values.astype(float)

## Visualize time-resolved vector flow learned with dynamo 

In [ ]:
dyn.tl.cell_velocities(adata, basis='umap_ori')

dyn.pl.streamline_plot(adata, color='cell_type', basis='umap_ori')


In [ ]:
dyn.pl.streamline_plot(adata, color='cell_cycle_phase', basis='umap_ori')


In [ ]:
adata.var_names[adata.var.use_for_transition][:5]

In [ ]:
dyn.pl.phase_portraits(adata, genes=['Brat1', 'Ccnd2', 'Ckmt1', 'Pdgfb', 'Gpa33'],
                       color='som_cluster_id', basis='umap_ori')


## Animate intestine organoid differentiation

In [ ]:
dyn.vf.VectorField(adata, basis='umap_ori')


In [ ]:
progenitor = adata.obs_names[adata.obs.cell_type == 'Stem cells']
len(progenitor)

In [ ]:
np.random.seed(19491001)

from matplotlib import animation
info_genes = adata.var_names[adata.var.use_for_transition]
dyn.pd.fate(adata, basis='umap_ori', init_cells=progenitor[:100], interpolation_num=100,  direction='forward',
   inverse_transform=False, average=False)


In [ ]:
%%capture
import matplotlib.pyplot as plt 

fig, ax = plt.subplots()
ax = dyn.pl.topography(adata, basis='umap_ori', color='cell_type', ax=ax, save_show_or_return='return',  figsize=(24, 24))
ax.set_aspect(0.8)


In [ ]:
%%capture
adata.obs['time'] = adata.obs.time.astype('float')
instance = dyn.mv.StreamFuncAnim(adata=adata, basis='umap_ori', color='cell_type', ax=ax)